In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt

# Prepare the dataset

Load the cat and dog dataset, splitting 80% for the training set and 20% for the test set.

In [4]:
# Create a autograph pre-processing function to resize and normalize an image
### START CODE HERE ###
@tf.function
def map_fn(img,label):
    image_height = 200
    image_width = 200
### START CODE HERE ###
    # resize the image
    img = tf.image.resize(img,[image_height, image_width])
    # normalize the image
    img /= 255
### END CODE HERE
    return img, label

In [6]:
base_dir = 'data_train/train/'

batch_size = 32
train_datagen = image_dataset_from_directory(base_dir,
                                                  image_size=(200,200),
                                                  subset='training',
                                                  seed = 1,
                                                 validation_split=0.1,
                                                  batch_size= 32)

len_train=train_datagen.cardinality().numpy()

train_datagen=train_datagen.map(map_fn)

test_datagen = image_dataset_from_directory(base_dir,
                                                  image_size=(200,200),
                                                  subset='validation',
                                                  seed = 1,
                                                 validation_split=0.1,
                                                  batch_size= 32)



Found 25000 files belonging to 2 classes.
Using 22500 files for training.
Found 25000 files belonging to 2 classes.
Using 2500 files for validation.


In [16]:
test_datagen=test_datagen.map(map_fn)

test_dir='data_test/'

ttt_datagen = image_dataset_from_directory(test_dir,
                                                  image_size=(200,200),
                                                  subset=None,
                                                  seed = 1,
                                                  batch_size= 32)

Found 12500 files belonging to 1 classes.


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 64)       0

In [8]:
def set_adam_optimizer():
    # Define the adam optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    return optimizer

def set_Binary_cat_crossentropy_loss():
    # Define object oriented metric of Sparse categorical crossentropy for train and val loss
    train_loss =  tf.keras.losses.BinaryCrossentropy()
    val_loss =  tf.keras.losses.BinaryCrossentropy()
    return train_loss, val_loss

def set_Binary_cat_crossentropy_accuracy():
    # Define object oriented metric of Sparse categorical accuracy for train and val accuracy
    train_accuracy = tf.keras.metrics.BinaryAccuracy()
    val_accuracy = tf.keras.metrics.BinaryAccuracy()
    return train_accuracy, val_accuracy


optimizer = set_adam_optimizer()
train_loss, val_loss = set_Binary_cat_crossentropy_loss()
train_accuracy, val_accuracy =set_Binary_cat_crossentropy_accuracy()

# Define the training loop

In [9]:
EPOCHS = 5

# Custom training step
def train_one_step(model, optimizer, x, y, train_loss, train_accuracy):
    '''
    Trains on a batch of images for one step.
    
    Args:
        model (keras Model) -- image classifier
        optimizer (keras Optimizer) -- optimizer to use during training
        x (Tensor) -- training images
        y (Tensor) -- training labels
        train_loss (keras Loss) -- loss object for training
        train_accuracy (keras Metric) -- accuracy metric for training
    '''
    with tf.GradientTape() as tape:
    ### START CODE HERE ###
        # Run the model on input x to get predictions
        predictions = model(x)
        # Compute the training loss using `train_loss`, passing in the true y and the predicted y
        loss = train_loss(y_true = y, y_pred = predictions)

    # Using the tape and loss, compute the gradients on model variables using tape.gradient
    grads = tape.gradient(loss, model.trainable_weights)
    
    # Zip the gradients and model variables, and then apply the result on the optimizer
    optimizer.apply_gradients(zip(grads, model.trainable_weights))


    # Call the train accuracy object on ground truth and predictions
    train_accuracy(y, predictions)
    ### END CODE HERE
    return loss

# Define the 'train' function

In [12]:
@tf.function
def train(model, optimizer, epochs, train_ds, train_loss, train_accuracy, valid_ds, val_loss, val_accuracy):
    '''
    Performs the entire training loop. Prints the loss and accuracy per step and epoch.
    
    Args:
        model (keras Model) -- image classifier
        optimizer (keras Optimizer) -- optimizer to use during training
        epochs (int) -- number of epochs
        train_ds (tf Dataset) -- the train set containing image-label pairs
        train_loss (keras Loss) -- loss function for training
        train_accuracy (keras Metric) -- accuracy metric for training
        valid_ds (Tensor) -- the val set containing image-label pairs
        val_loss (keras Loss) -- loss object for validation
        val_accuracy (keras Metric) -- accuracy metric for validation
    '''
    step = 0
    loss = 0.0
    for epoch in range(epochs):
        for x, y in train_ds:
            # training step number increments at each iteration
            step += 1
            # Run one training step by passing appropriate model parameters
            # required by the function and finally get the loss to report the results
            loss = train_one_step(model, optimizer, x, y, train_loss, train_accuracy)
            # Use tf.print to report your results.
            # Print the training step number, loss and accuracy
            tf.print('Step', step, 
                   ': train loss', loss, 
                   '; train accuracy', train_accuracy.result())

       
        for x, y in valid_ds:
            # Call the model on the batches of inputs x and get the predictions
            y_pred = model(x)
            loss = val_loss(y, y_pred)
            val_accuracy(y, y_pred)
        
        # Print the validation loss and accuracy
        ### START CODE HERE ###
        tf.print('val loss', loss, '; val accuracy', val_accuracy.result())
        ### END CODE HERE ###

In [13]:
train(model, optimizer, EPOCHS, train_datagen, train_loss, 
      train_accuracy, test_datagen, val_loss, val_accuracy)

2023-08-15 15:23:20.259698: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_47' with dtype resource
	 [[{{node Placeholder/_47}}]]
2023-08-15 15:23:20.261792: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_33' with dtype resource
	 [[{{node Placeholder/_33}}]]


Step 1 : train loss 0.694430113 ; train accuracy 0.5
Step 2 : train loss 2.97861719 ; train accuracy 0.515625
Step 3 : train loss 1.59164906 ; train accuracy 0.479166657
Step 4 : train loss 1.04870331 ; train accuracy 0.484375
Step 5 : train loss 0.683485746 ; train accuracy 0.5
Step 6 : train loss 0.678450465 ; train accuracy 0.505208313
Step 7 : train loss 0.764632583 ; train accuracy 0.5
Step 8 : train loss 0.677611828 ; train accuracy 0.5078125
Step 9 : train loss 0.741294265 ; train accuracy 0.5
Step 10 : train loss 0.657588661 ; train accuracy 0.5125
Step 11 : train loss 0.669606805 ; train accuracy 0.522727251
Step 12 : train loss 0.701454043 ; train accuracy 0.518229187
Step 13 : train loss 0.728357613 ; train accuracy 0.492788464
Step 14 : train loss 0.688043 ; train accuracy 0.502232134
Step 15 : train loss 0.689015865 ; train accuracy 0.50625
Step 16 : train loss 0.718540311 ; train accuracy 0.5
Step 17 : train loss 0.72539854 ; train accuracy 0.492647052
Step 18 : train los

Step 136 : train loss 0.603136182 ; train accuracy 0.568933845
Step 137 : train loss 0.651767671 ; train accuracy 0.568658769
Step 138 : train loss 0.645591855 ; train accuracy 0.568161249
Step 139 : train loss 0.690283656 ; train accuracy 0.567670882
Step 140 : train loss 0.593984365 ; train accuracy 0.568080366
Step 141 : train loss 0.615713596 ; train accuracy 0.569148958
Step 142 : train loss 0.580444455 ; train accuracy 0.570642591
Step 143 : train loss 0.616890371 ; train accuracy 0.571022749
Step 144 : train loss 0.583389223 ; train accuracy 0.572265625
Step 145 : train loss 0.575936794 ; train accuracy 0.573706925
Step 146 : train loss 0.598599 ; train accuracy 0.573630154
Step 147 : train loss 0.797582269 ; train accuracy 0.573767
Step 148 : train loss 0.605107367 ; train accuracy 0.574535489
Step 149 : train loss 0.631348729 ; train accuracy 0.574874163
Step 150 : train loss 0.611007929 ; train accuracy 0.575833321
Step 151 : train loss 0.64762342 ; train accuracy 0.576158941

Step 268 : train loss 0.70500803 ; train accuracy 0.620452404
Step 269 : train loss 0.429950714 ; train accuracy 0.621282518
Step 270 : train loss 0.419816434 ; train accuracy 0.62199074
Step 271 : train loss 0.486725956 ; train accuracy 0.622347772
Step 272 : train loss 0.711005211 ; train accuracy 0.622702181
Step 273 : train loss 0.655557811 ; train accuracy 0.622939587
Step 274 : train loss 0.543528318 ; train accuracy 0.623175204
Step 275 : train loss 0.461531162 ; train accuracy 0.623636365
Step 276 : train loss 0.587907493 ; train accuracy 0.623981
Step 277 : train loss 0.584604681 ; train accuracy 0.624210298
Step 278 : train loss 0.522371769 ; train accuracy 0.624662757
Step 279 : train loss 0.626476049 ; train accuracy 0.624551952
Step 280 : train loss 0.620231271 ; train accuracy 0.624665201
Step 281 : train loss 0.579866052 ; train accuracy 0.624888778
Step 282 : train loss 0.549369216 ; train accuracy 0.625110805
Step 283 : train loss 0.659333467 ; train accuracy 0.6252208

Step 400 : train loss 0.613421142 ; train accuracy 0.645312488
Step 401 : train loss 0.432694823 ; train accuracy 0.645729423
Step 402 : train loss 0.600327492 ; train accuracy 0.646066546
Step 403 : train loss 0.460427105 ; train accuracy 0.646479547
Step 404 : train loss 0.50173229 ; train accuracy 0.646735787
Step 405 : train loss 0.585996747 ; train accuracy 0.646682084
Step 406 : train loss 0.517908037 ; train accuracy 0.646782637
Step 407 : train loss 0.502199888 ; train accuracy 0.647113
Step 408 : train loss 0.55190146 ; train accuracy 0.64728862
Step 409 : train loss 0.491531372 ; train accuracy 0.647692561
Step 410 : train loss 0.712710142 ; train accuracy 0.647560954
Step 411 : train loss 0.487062305 ; train accuracy 0.647886276
Step 412 : train loss 0.523731351 ; train accuracy 0.648134112
Step 413 : train loss 0.672944546 ; train accuracy 0.647926748
Step 414 : train loss 0.413790405 ; train accuracy 0.64839977
Step 415 : train loss 0.781009912 ; train accuracy 0.648268044

...

Step 3201 : train loss 0.0910962075 ; train accuracy 0.827023089
Step 3202 : train loss 0.160106987 ; train accuracy 0.8270576
Step 3203 : train loss 0.120451644 ; train accuracy 0.827092111
Step 3204 : train loss 0.0630299747 ; train accuracy 0.827136397
Step 3205 : train loss 0.0692879 ; train accuracy 0.827190399
Step 3206 : train loss 0.0894645825 ; train accuracy 0.827234566
Step 3207 : train loss 0.123852052 ; train accuracy 0.827269
Step 3208 : train loss 0.0443070531 ; train accuracy 0.8273229
Step 3209 : train loss 0.0743297189 ; train accuracy 0.827357292
Step 3210 : train loss 0.0389650613 ; train accuracy 0.827411115
Step 3211 : train loss 0.112290144 ; train accuracy 0.827445447
Step 3212 : train loss 0.147958 ; train accuracy 0.82747972
Step 3213 : train loss 0.0791823491 ; train accuracy 0.827523768
Step 3214 : train loss 0.0196352415 ; train accuracy 0.827577472
Step 3215 : train loss 0.0173390526 ; train accuracy 0.827631176
Step 3216 : train loss 0.00784850121 ; train

Step 3329 : train loss 0.0886396319 ; train accuracy 0.832262039
Step 3330 : train loss 0.0580586419 ; train accuracy 0.832303107
Step 3331 : train loss 0.0945925936 ; train accuracy 0.832344115
Step 3332 : train loss 0.150173038 ; train accuracy 0.832375705
Step 3333 : train loss 0.182136953 ; train accuracy 0.832397878
Step 3334 : train loss 0.0175513253 ; train accuracy 0.832448184
Step 3335 : train loss 0.0899255425 ; train accuracy 0.832489133
Step 3336 : train loss 0.0389707312 ; train accuracy 0.83253938
Step 3337 : train loss 0.136702374 ; train accuracy 0.832561493
Step 3338 : train loss 0.147825435 ; train accuracy 0.832592964
Step 3339 : train loss 0.114770532 ; train accuracy 0.832624435
Step 3340 : train loss 0.0481847376 ; train accuracy 0.832665205
Step 3341 : train loss 0.25011611 ; train accuracy 0.832668543
Step 3342 : train loss 0.033263538 ; train accuracy 0.83271867
Step 3343 : train loss 0.0573752336 ; train accuracy 0.83275938
Step 3344 : train loss 0.0127503704 

Step 3457 : train loss 0.0174900796 ; train accuracy 0.837284625
Step 3458 : train loss 0.0976719111 ; train accuracy 0.837322712
Step 3459 : train loss 0.0725047588 ; train accuracy 0.83736074
Step 3460 : train loss 0.0872228146 ; train accuracy 0.837389708
Step 3461 : train loss 0.112674244 ; train accuracy 0.837418675
Step 3462 : train loss 0.0345366076 ; train accuracy 0.837465644
Step 3463 : train loss 0.114586815 ; train accuracy 0.837503612
Step 3464 : train loss 0.1394227 ; train accuracy 0.83752346
Step 3465 : train loss 0.0841216445 ; train accuracy 0.837570429
Step 3466 : train loss 0.0834489465 ; train accuracy 0.837599277
Step 3467 : train loss 0.0109985657 ; train accuracy 0.837646186
Step 3468 : train loss 0.0751453936 ; train accuracy 0.837675
Step 3469 : train loss 0.148307979 ; train accuracy 0.837703824
Step 3470 : train loss 0.0869513527 ; train accuracy 0.837732613
Step 3471 : train loss 0.0251593068 ; train accuracy 0.837779403
Step 3472 : train loss 0.0233055335 

2023-08-15 16:10:10.860913: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype resource
	 [[{{node Placeholder/_20}}]]
2023-08-15 16:10:10.862029: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_25' with dtype resource
	 [[{{node Placeholder/_25}}]]


val loss 0.989784122 ; val accuracy 0.79704
